In [ ]:
# CLEAR OUTPUT BEFORE COMMITTING!!!

In [ ]:
import sqlite3
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

plt.rcParams.update({'font.size': 32})
sns.set()

In [ ]:
conn = sqlite3.connect("db.db")
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'", conn)

In [ ]:
projects = pd.read_sql("""
    SELECT *
    FROM SQ_Project
""", conn)

issues = pd.read_sql("""
    SELECT *
    FROM SQ_Issue
""", conn)

projects = projects.set_index("id")
issues = issues.set_index("id")

In [ ]:
conn.close()

In [ ]:
projects = projects[projects.index != "f22_hw3_0e83d1dddd1c3468d23a033da13e34bfdbd3b09e3c129d2d4fe4044acaf0bbc6"]
issues = issues[issues["project_id"] != "f22_hw3_0e83d1dddd1c3468d23a033da13e34bfdbd3b09e3c129d2d4fe4044acaf0bbc6"]

projects = projects[projects.index != "f22_hw6_bf07896588ac21f4471e818c5a7475898cf80e609f73a93134d9f6da5a6e3e38"]
issues = issues[issues["project_id"] != "f22_hw6_bf07896588ac21f4471e818c5a7475898cf80e609f73a93134d9f6da5a6e3e38"]

In [ ]:
megaTable = issues.join(projects, on="project_id")

In [ ]:
len(projects.groupby("student").count()["hw"])

In [ ]:
projects.columns

In [ ]:
issues.columns

In [ ]:
megaTable.columns

In [ ]:
megaTable[megaTable["project_id"] == "f22_hw3_6542b43b5548d24d8632ae0ef3e57e1a3da38c79bf830605c692c76b8f3c24d7"]['message']

In [ ]:
megaTable[megaTable["project_id"] == "f22_hw3_TEMPLATE"]['message']

In [ ]:
megaTable

In [ ]:
print(len(megaTable[megaTable["severity"] == "INFO"]))
print(len(megaTable[megaTable["severity"] == "MINOR"]))
print(len(megaTable[megaTable["severity"] == "MAJOR"]))
print(len(megaTable[megaTable["severity"] == "CRITICAL"]))
print(len(megaTable[megaTable["severity"] == "BLOCKER"]))

In [ ]:
megaTable.groupby(["type", "severity"]) \
    .count()["project_id"]

In [ ]:
makeup = megaTable.groupby(["type", "severity"]).count()["project_id"].to_frame()
# makeup['type'] = makeup.index
makeup = makeup.reset_index(level=0)
makeup = makeup.reset_index(level=0)
makeup = makeup.rename(columns={'project_id': 'count'})
makeup

In [ ]:
megaTable.value_counts("severity", normalize=True)

In [ ]:
def fillCol(df, sev, typ):
    if(not sev in df[df["type"] == typ]["severity"].tolist()):
        df = df.append({
            "severity": sev,
            "perc": 0,
            "type": typ
        }, ignore_index=True)
    return df

In [ ]:
# Cole sucks at Pandas

SEV_ORDER = ["BLOCKER", "CRITICAL", "MAJOR", "MINOR", "INFO"]

percBugg = megaTable[megaTable["type"] == "BUG"].value_counts("severity", normalize=True)
percSmel = megaTable[megaTable["type"] == "CODE_SMELL"].value_counts("severity", normalize=True)
percVuln = megaTable[megaTable["type"] == "VULNERABILITY"].value_counts("severity", normalize=True)

percBuggDf = percBugg.to_frame("perc").reset_index()
percSmelDf = percSmel.to_frame("perc").reset_index()
percVulnDf = percVuln.to_frame("perc").reset_index()

percBuggDf['type'] = 'BUG'
percSmelDf['type'] = 'CODE_SMELL'
percVulnDf['type'] = 'VULNERABILITY'

megaPerc = pd.concat([percBuggDf, percSmelDf, percVulnDf]).reset_index().drop("index", axis=1)

for i in ["BUG", "CODE_SMELL", "VULNERABILITY"]:
    for j in SEV_ORDER:
        megaPerc = fillCol(megaPerc, j, i);

megaPerc = megaPerc.append({
    "severity": "INFO",
    "perc": len(megaTable[megaTable["severity"] == "INFO"]) / len(megaTable),
    "type": "ALL"
}, ignore_index=True)

megaPerc = megaPerc.append({
    "severity": "MINOR",
    "perc": len(megaTable[megaTable["severity"] == "MINOR"]) / len(megaTable),
    "type": "ALL"
}, ignore_index=True)

megaPerc = megaPerc.append({
    "severity": "MAJOR",
    "perc": len(megaTable[megaTable["severity"] == "MAJOR"]) / len(megaTable),
    "type": "ALL"
}, ignore_index=True)

megaPerc = megaPerc.append({
    "severity": "CRITICAL",
    "perc": len(megaTable[megaTable["severity"] == "CRITICAL"]) / len(megaTable),
    "type": "ALL"
}, ignore_index=True)

megaPerc = megaPerc.append({
    "severity": "BLOCKER",
    "perc": len(megaTable[megaTable["severity"] == "BLOCKER"]) / len(megaTable),
    "type": "ALL"
}, ignore_index=True)

megaPerc['perc'] = megaPerc['perc'] * 100

percInfo = megaTable[megaTable["severity"] == "INFO"].value_counts("type", normalize=True)
percMino = megaTable[megaTable["severity"] == "MINOR"].value_counts("type", normalize=True)
percMajo = megaTable[megaTable["severity"] == "MAJOR"].value_counts("type", normalize=True)
percCrit = megaTable[megaTable["severity"] == "CRITICAL"].value_counts("type", normalize=True)
percBloc = megaTable[megaTable["severity"] == "BLOCKER"].value_counts("type", normalize=True)

percInfoDf = percInfo.to_frame("perc").reset_index()
percMinoDf = percMino.to_frame("perc").reset_index()
percMajoDf = percMajo.to_frame("perc").reset_index()
percBlocDf = percCrit.to_frame("perc").reset_index()
percCritDf = percBloc.to_frame("perc").reset_index()

percInfoDf['severity'] = 'INFO'
percMinoDf['severity'] = 'MINOR'
percMajoDf['severity'] = 'MAJOR'
percCritDf['severity'] = 'CRITICAL'
percBlocDf['severity'] = 'BLOCKER'

megaPercOt = pd.concat([percInfoDf, percMinoDf, percMajoDf, percCritDf, percBlocDf]).reset_index().drop("index", axis=1)

for i in ["BUG", "CODE_SMELL", "VULNERABILITY"]:
    for j in SEV_ORDER:
        megaPercOt = fillCol(megaPercOt, j, i);

megaPercOt = megaPercOt.append({
    "severity": "ALL",
    "perc": len(megaTable[megaTable["type"] == "BUG"]) / len(megaTable),
    "type": "BUG"
}, ignore_index=True)

megaPercOt = megaPercOt.append({
    "severity": "ALL",
    "perc": len(megaTable[megaTable["type"] == "CODE_SMELL"]) / len(megaTable),
    "type": "CODE_SMELL"
}, ignore_index=True)

megaPercOt = megaPercOt.append({
    "severity": "ALL",
    "perc": len(megaTable[megaTable["type"] == "VULNERABILITY"]) / len(megaTable),
    "type": "VULNERABILITY"
}, ignore_index=True)

megaPercOt['perc'] = megaPercOt['perc'] * 100

In [ ]:
megaPerc.sort_values(["type", "severity"]).set_index(["type", "severity"])["perc"]

In [ ]:
ax = megaPerc.set_index(["severity", "type"])["perc"] \
    .loc[SEV_ORDER] \
    .unstack('severity') \
    .loc[["ALL", "VULNERABILITY", "CODE_SMELL", "BUG"]] \
    .sort_index(key=lambda l: l.str.replace("I", "ZI"), axis=1) \
    .plot.bar(color=["darkred", "salmon", "sandybrown", "khaki", "skyblue"], stacked=True)

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Issue Severity")
ax.set_xlabel("Issue Type", fontsize=20)
ax.set_ylabel("% Makeup", fontsize=20)
ax.set_title("% Makeup by Issue Type", fontweight='bold', fontsize=26)
plt.xticks(rotation=45)

In [ ]:
ax = megaPercOt.set_index(["type", "severity"])["perc"] \
    .loc[["CODE_SMELL", "BUG", "VULNERABILITY"]] \
    .unstack('type') \
    .loc[["ALL", "BLOCKER", "CRITICAL", "MAJOR", "MINOR", "INFO"]] \
    .sort_index(key=lambda l: l.str.replace("V", "AV"), axis=1) \
    .plot.bar(stacked=True, color=["salmon", "khaki", "skyblue"])

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Issue Type")
ax.set_xlabel("Issue Severity", fontsize=20)
ax.set_ylabel("% Makeup", fontsize=20)
ax.set_title("% Makeup by Issue Severity", fontweight="bold", fontsize=26)
plt.xticks(rotation=45)

In [ ]:
megaPerc

In [ ]:
megaTable.groupby(["type", "severity"]).count()["project_id"]

In [ ]:
SEV_ORDER = ["BLOCKER", "CRITICAL", "MAJOR", "MINOR", "INFO"]

ax = megaTable.groupby(["type", "severity"]) \
    .count()["project_id"] \
    .unstack('type') \
    .loc[SEV_ORDER] \
    .sort_index(key=lambda l: l.str.replace("V", "AV"), axis=1) \
    .plot.bar(color=["salmon", "khaki", "skyblue"])

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Issue Type")
ax.set_xlabel("Issue Severity", fontsize=20)
ax.set_ylabel("# of Issues", fontsize=20)
ax.set_title("# of Issues by Issue Severity", fontweight="bold", fontsize=26)
plt.xticks(rotation=45)

In [ ]:
TYP_ORDER = ["VULNERABILITY", "CODE_SMELL", "BUG"]
ax = megaTable.groupby(["severity", "type"]) \
    .count()["project_id"] \
    .unstack('severity') \
    .loc[TYP_ORDER] \
    .plot.bar(color=["darkred", "salmon", "sandybrown", "khaki", "skyblue"])

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Issue Severity")
ax.set_xlabel("Issue Type", fontsize=20)
ax.set_ylabel("# of Issues", fontsize=20)
ax.set_title("# of Issues by Issue Type", fontweight="bold", fontsize=26)
plt.xticks(rotation=45)

In [ ]:
megaTable.groupby(["type", "severity"]).count()["project_id"]

In [ ]:
megaTable.groupby("student").count()["project_id"]

In [ ]:
megaTable[megaTable["student"] == "10b24926a66942e1d84d1fe81805f611fed3af8923779167c96162d92ab88c2a"] \
    .groupby("project_id").count()["student"].sort_values(ascending=False)

In [ ]:
vuln_issues = megaTable[megaTable["type"] == "VULNERABILITY"]
smel_issues = megaTable[megaTable["type"] == "CODE_SMELL"]
bugg_issues = megaTable[megaTable["type"] == "BUG"]

In [ ]:
breakdown = pd.Series([len(vuln_issues), len(smel_issues), len(bugg_issues)], index=["Vulnerability", "Code Smell", "Bug"])

In [ ]:
breakdown.plot.pie(title="SAST Issue Classification", ylabel="", explode=(0.1, 0.1, 0.1), colors=['salmon', 'lightgreen', 'skyblue'], startangle=90)

In [ ]:
len(vuln_issues)

In [ ]:
len(bugg_issues)

In [ ]:
len(smel_issues)

In [ ]:
def plot_pie_breakdown(df, title):
    df_counted = df.groupby("severity").count()["project_id"]
    ord_df_count = pd.Series([\
         df_counted["INFO"] if  "INFO" in df_counted else 0, \
         df_counted["MINOR"] if  "MINOR" in df_counted else 0, \
         df_counted["MAJOR"] if  "MAJOR" in df_counted else 0, \
         df_counted["CRITICAL"] if  "CRITICAL" in df_counted else 0, \
         df_counted["BLOCKER"] if  "BLOCKER" in df_counted else 0, \
        ], index=["Info", "Minor", "Major", "Critical", "Blocker"])
    ord_df_count.plot.pie(startangle=90, title=title, explode=(0.1, 0.1, 0.1, 0.1, 0.1), ylabel="", colors=["skyblue", "khaki", "sandybrown", "salmon", "darkred"])

In [ ]:
plot_pie_breakdown(megaTable, "SAST Issue Severity (All)")

In [ ]:
plot_pie_breakdown(vuln_issues, "Vulnerability Issue Severity")

In [ ]:
plot_pie_breakdown(bugg_issues, "Bug Issue Severity")

In [ ]:
plot_pie_breakdown(smel_issues, "Code Smell Issue Severity")

In [ ]:
megaTable[(megaTable['rule'] == 'javascript:S1135') & (megaTable['student'] == '6ff85833fc31e8bab5bad382ca8d79d2fb245f175d4fa06721d5655ef51f3e07')]


In [ ]:
smel = smel_issues.groupby("rule").count()["project_id"].sort_values(ascending=False)
print(sum(smel[5:]))
print(len(smel) - 5)
smel[:5]

In [ ]:
bugg = bugg_issues.groupby("rule").count()["project_id"].sort_values(ascending=False)
print(sum(bugg[5:]))
print(len(bugg) - 5)
bugg[:5]

In [ ]:
vuln_issues.groupby("rule").count()["project_id"].sort_values(ascending=False)

In [ ]:
sast_findings = megaTable.groupby("student").count()["project_id"]
ax = sast_findings.hist(bins=30)
ax.set_xlabel("# of Issues")
ax.set_ylabel("# of Students")
ax.set_title("# of Issues by # of Students")

In [ ]:
vuln_issues[vuln_issues["student"] == "0385977ba8d1b3161e3377e46dafabb769421f3e963bc7d4ffd983b5eb9f1058"]

In [ ]:
vuln_findings = vuln_issues.groupby("student").count()["project_id"]
vuln_findings

In [ ]:
# new_vulns = (vuln_issues.groupby("student").count()["project_id"] - 67)
# sorted(new_vulns[new_vulns != 0])

In [ ]:
stats.normaltest(sast_findings)

In [ ]:
# https://support.sisense.com/kb/en/article/test-for-normal-distribution-of-data-with-python
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""

    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

In [ ]:
x, y = ecdf(sast_findings)
plt.figure()
plt.plot(x, y, marker=".", linestyle="none")
plt.xlabel("# of Issues")
plt.ylabel("Cumulative Distribution Function")
samples = np.random.normal(np.mean(sast_findings), np.std(sast_findings), size=10000)
x_theor, y_theor = ecdf(samples)
plt.plot(x_theor, y_theor)
plt.legend(('Empirical Data', 'Normal Distribution'), loc='lower right')
plt.title("Empirical vs Normal Distribution of Data")

In [ ]:
# # x, y = ecdf(sast_findings)
# # plt.figure()
# # plt.plot(x, y, marker=".", linestyle="none")
# # plt.xlabel("# of SAST Findings")
# plt.ylabel("Cumulative Distribution Function")
# samples = np.random.poisson(np.mean(sast_findings), size=10000)
# y_theor = stats.poisson.cdf(samples, np.mean(sast_findings))
# y_theor = stats.poisson.pmf(samples, np.mean(sast_findings))
# print(y_theor)
# theor = [(x, y) for x, y in sorted(zip(samples, y_theor))]
# x_theor = [i[0] for i in theor]
# y_theor = [i[1] for i in theor]
# plt.plot(x_theor, y_theor)
# plt.legend(('Empirical Data', 'Poission Distribution'), loc='lower right')
# plt.title("Empirical vs Poission Distribution of Data")

In [ ]:
sast_findings.mean() / 8

In [ ]:
sast_findings.std() / 8

In [ ]:
sast_findings.values